#Setup Detectron 2 (Libraries to install)


In [1]:
!pip install pyyaml==5.1
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 274 kB 5.2 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=a6bb5322c42d0b326db1395482544e4104bf5f0881de58918174e96590dad1a7
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 17 kB/s 
     |████████████████████████████████| 12.8 MB 23.7 MB/s 
     |████████████████████████████████| 7.6 MB 4.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu

In [2]:
import torch 
print(torch.__version__)
print(torch.cuda.is_available())


1.7.1+cu101
True


In [3]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
     |████████████████████████████████| 6.1 MB 952 kB/s 
     |████████████████████████████████| 74 kB 1.5 MB/s 
     |████████████████████████████████| 147 kB 9.1 MB/s 
     |████████████████████████████████| 50 kB 4.8 MB/s 
     |████████████████████████████████| 130 kB 34.0 MB/s 
     |████████████████████████████████| 749 kB 32.4 MB/s 
     |████████████████████████████████| 843 kB 34.3 MB/s 
     |████████████████████████████████| 112 kB 32.1 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220414-py3-none-any.whl size=61211 sha256=2c822eccac907f8307305a1502d874cd954d8d822a59e1cd4cc9bd7145174ee2
  Stored in directory: /root/.cache/pip/wheels/df/f4/b8/7b5df8b6722f4c72315ce70c31aa693e00cef6a5056149bd28
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=9933ad1b6fcc4ba5fb78376e90d3ae354c5730fc54587101ebf4d2e8e5cb492

#Import Detectron2





In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import cv2
from google.colab.patches import cv2_imshow

import numpy as np



```
# This is formatted as code
```

#DETECTOR MODEL WITH DETECTRON 2

In [ ]:
all_image_data_list = []
seg_info = []
thing_names = []
stuff_names = []

class Detector:

  def __init__(self):
    self.cfg = get_cfg()

    #load model config and pretrained model
    self.cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
    self.cfg.MODEL.WEIGHTS = (model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))

    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
    self.cfg.MODEL.DEVICE = "cuda"
      
    self.predictor = DefaultPredictor(self.cfg)

    thing_names.extend(MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]).thing_classes)
    stuff_names.extend(MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]).stuff_classes)

  #get image function to apply segmentation 
  def onImage(self,imagePath):
    image = cv2.imread(imagePath)
    
    predictions, segmentInfo = self.predictor(image)["panoptic_seg"]


    viz = Visualizer(image[:,:,::-1], metadata = MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]), instance_mode = ColorMode.IMAGE)

    output = viz.draw_panoptic_seg_predictions(predictions.to("cpu"), segmentInfo)
  
    #cv2_imshow(output.get_image()[:,:,::-1]) #saving time and ram
    cv2.waitKey(0)

    seg_info.extend(segmentInfo)

#_______C0DE FOR EXTRACTING DETECTED OBJECTS LIST_______

   #GET THE OBJECT LIST
    things = []
    stuffs = []
    for i in range(len(seg_info)):
        exist_key = 'instance_id'
        if exist_key in seg_info[i]: 
          things.append(seg_info[i])
        else:
          stuffs.append(seg_info[i]) 
     
    #CATEGORY ID FOR THINGS CLASS
    category_id_things = [] 
    # to string using list comprehension
    things = ' '.join(map(str, things))
    import re
    # function to demonstrate removal of characters
    # which are not numbers and alphabets using re
    things = re.sub('[\W_]+', '', things)
    # get the category numbers
    regex = re.compile('categoryid([0-9]*)')
    category_id_things = regex.findall(things)
    category_id_things = list(dict.fromkeys(category_id_things))
    
    #CATEGORY ID FOR STUFF CLASS
    category_id_stuffs = [] 
    # to string using list comprehension
    stuffs = ' '.join(map(str, stuffs))
    import re
    # function to demonstrate removal of characters
    # which are not numbers and alphabets using re
    stuffs = re.sub('[\W_]+', '', stuffs)
    # get the category numbers
    regex = re.compile('categoryid([0-9]*)')
    category_id_stuffs = regex.findall(stuffs)
    category_id_stuffs = list(dict.fromkeys(category_id_stuffs))

    #map and show the object list
    list_of_obj = []
    for i in range(len(category_id_things)):
      a = int(category_id_things[i])
      list_of_obj.append(thing_names[a])
    for i in range(len(category_id_stuffs)):
      a = int(category_id_stuffs[i])
      list_of_obj.append(stuff_names[a])
    print(list_of_obj)

    #SAVE THE LIST IN CSV FILE PER INDEX
    all_image_data_list.append(list_of_obj)
    
    #clear the segment info list for new image input 
    seg_info.clear()

#------------------------------------------------------------------------------------------------''' 
detector = Detector()

model_final_cafdb1.pkl:  52%|█████▏    | 135M/261M [00:03<00:02, 44.3MB/s]

#LOAD INPUT IMAGES AND EXTRACT THE LIST OF DETECTED OBJECTS 

In [ ]:
from tqdm import tqdm
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/OCO DETECTION/COCO2017"
directory = os.path.join(BASE_DIR, '3rd') #folder name

for img_name in tqdm(os.listdir(directory)):
  #LOAD THE IMAGE FROM FILE
  img_path= directory + "/" + img_name
  detector.onImage(img_path)

Output hidden; open in https://colab.research.google.com to view.

#SAVE* OBJECT LISTS IN txt -> CSV
*#SAVE IN CSV FILE (single column object lists )*

In [ ]:
#SAVE IN TEXT FILE 

from pathlib import Path
my_file = Path("/content/drive/MyDrive/Colab Notebooks/OCO DETECTION/Detected Object Lists/6th.txt")

# check if directory exists
if my_file.is_file():
   with open (my_file, 'a') as my_file:
    for LIST in all_image_data_list:
       my_file.write(str(LIST) + '\n') 
else:
    print('File not available')

In [ ]:
#path
csv_path = "/content/drive/MyDrive/Colab Notebooks/OCO DETECTION/Detected Object Lists/6th.csv"
txt_path = "/content/drive/MyDrive/Colab Notebooks/OCO DETECTION/Detected Object Lists/6th.txt"
c_path = "/content/drive/MyDrive/Colab Notebooks/OCO DETECTION/Detected Object Lists/runtime errors/cleaned.txt"

In [ ]:
import re
uncleanText = open(txt_path).read()
cleanText = re.sub('[^A-Za-z,\n\s]+', '', uncleanText)
cleanText

'person, umbrella, sand, fence, sky, pavement, mountain, building\ncell phone, book, shelf, table, paper\nperson, bird, traffic light, bench, truck, handbag, tree, pavement\nperson, bowl, car, towel, tree, grass\nperson, cell phone, chair, window, building\nperson, cell phone, floor, rug\nteddy bear, vase, bottle, dining table, flower, shelf, wallwood, wall\nperson, cell phone, cup, table, wall\ncell phone, book, cardboard, table\nteddy bear, wall\nteddy bear\nperson, tree, grass, dirt\nteddy bear, frisbee, floorwood\nperson, chair, dog, floorwood, wallwood, window, cabinet, table, wall, rug\nteddy bear, person, umbrella, blanket, curtain, wall\nperson, wallwood, wall\nperson, teddy bear, house, wallstone, sky, mountain, building, rock\ncup, carrot, broccoli, bowl, table\nwine glass, person, potted plant, book, bottle, dining table, floorwood, shelf, wallbrick, ceiling, cabinet, wall\nperson, car, bicycle, potted plant, traffic light, road, sky, pavement, building\nperson, bottle, chai

In [ ]:
with open(c_path, 'w') as f:
    f.write(cleanText)

In [ ]:
#save in csv
import pandas as pd
  
# readinag given csv file
# and creating dataframe
dataframe1 = pd.read_csv(c_path,sep='delimiter', header=None)

# storing this dataframe in a csv file
dataframe1.to_csv(csv_path, 
                  header=False, index=False)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
'''
#SAVE IN CSV FILE (per column object name )

import pandas as pd
import csv

# Convert to dataframe
df_Data = pd.DataFrame(all_image_data_list)

# Export to csv file
df_Data.to_csv("/content/drive/MyDrive/Colab Notebooks/OCO DETECTION/Detected Object Lists/6th.csv")
'''